In [2]:
# app_gradio.py

import gradio as gr
import torch
from transformers import SeamlessM4Tv2ForSpeechToText, AutoProcessor
import torchaudio

device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "facebook/seamless-m4t-v2-large"
processor = AutoProcessor.from_pretrained(model_id)
model = SeamlessM4Tv2ForSpeechToText.from_pretrained(model_id).to(device)

def translate_speech_to_english(audio_file):
    speech_array, sampling_rate = torchaudio.load(audio_file)
    if sampling_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
        speech_array = resampler(speech_array)
    speech_array = speech_array.squeeze().numpy()

    inputs = processor(audios=speech_array, sampling_rate=16000, return_tensors="pt")
    for k in inputs:
        inputs[k] = inputs[k].to(device)

    translated_tokens = model.generate(**inputs, tgt_lang="eng")
    translated_text = processor.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    return translated_text
    print("🚀 Loading complete. Launching Gradio UI...")

iface = gr.Interface(
    fn=translate_speech_to_english,
    inputs=gr.Audio(type="filepath", label="Upload Audio"),
    outputs="text",
    title="SeamlessM4T Speech Translator",
    description="Upload an audio file (Hindi, Marathi, Punjabi, etc.) and get English translation using Seamless M4T."
)

if __name__ == "__main__":
    iface.launch()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://63a70c7d43a98f793e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
